In [1]:
%pip install numpy matplotlib scapy[basic] tqdm

  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7e/07/4b361d6d0f4e08942575f83a11d33f36897e1aae4279046606dd1808778a/matplotlib-3.1.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cd/80/5bb262050dd2f30f8819626b7c92339708fe2ed7bd5554c8193b4487b367/tqdm-4.42.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5d/bc/1e58593167fade7b544bfe9502a26dc860940a79ab306e651e7f13be68c2/pyparsing-2.4.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://f

In [2]:
from tqdm import tqdm
from scapy.all import PcapReader


def accumulate(file, *accumulators):
    with PcapReader(file) as reader, tqdm(unit='packets', desc=file) as pbar:
        for packet in reader:
            for accumulator in accumulators:
                accumulator(packet)
            pbar.update()
        return accumulators

In [40]:
import random
import numpy as np
from scapy.layers.dot11 import Dot11ProbeReq


class ProbeReqAcc(list):
    TYPE = ('timestamp', np.float), ('mac', np.string_, 17)
    
    def __call__(self, packet):
        if packet.haslayer(Dot11ProbeReq):
            values = packet.time, packet.addr2
            self.append(values)
            
    def as_numpy_array(self):
        return np.array(self, dtype=np.dtype([*self.TYPE]))


class RandProbeReqSampler(list):
    PROBABILITY = 0.001
    
    def __call__(self, packet):
        if packet.haslayer(Dot11ProbeReq) and random.random() <= self.PROBABILITY:
            self.append(packet)

In [41]:
commute_16122019, commute_16122019_samples = accumulate('capture-0-a4.pcap', ProbeReqAcc(), RandProbeReqSampler())
commute_17122019, commute_17122019_samples = accumulate('capture-1-a4.pcap', ProbeReqAcc(), RandProbeReqSampler())
commute_18122019, commute_18122019_samples = accumulate('capture-2-a4.pcap', ProbeReqAcc(), RandProbeReqSampler())

capture-0-a4.pcap: 57279packets [01:05, 873.86packets/s] 
capture-1-a4.pcap: 67268packets [01:17, 863.96packets/s] 
capture-2-a4.pcap: 59223packets [01:09, 857.95packets/s] 


In [42]:
def duration(data):
    timestamps = data[:]['timestamp']
    return np.amax(timestamps) - np.amin(timestamps)


print(duration(commute_16122019.as_numpy_array()))
print(duration(commute_17122019.as_numpy_array()))
print(duration(commute_18122019.as_numpy_array()))

5516.539717912674
5496.147868871689
4705.994265079498


In [47]:
commute_16122019_samples[0].show()


###[ RadioTap dummy ]### 
  version   = 0
  pad       = 0
  len       = 18
  present   = Flags+Rate+Channel+dBm_AntSignal+Antenna+RXFlags
  Flags     = 
  Rate      = 2
  ChannelFrequency= 2462
  ChannelFlags= CCK+2GHz
  dBm_AntSignal= -42dBm
  Antenna   = 1
  RXFlags   = 
  notdecoded= ''
###[ 802.11 ]### 
     subtype   = 4
     type      = Management
     proto     = 0
     FCfield   = 
     ID        = 0
     addr1     = ff:ff:ff:ff:ff:ff
     addr2     = f6:b1:53:4c:01:e8
     addr3     = ff:ff:ff:ff:ff:ff
     SC        = 15760
###[ 802.11 Probe Request ]### 
###[ 802.11 Information Element ]### 
           ID        = SSID
           len       = 0
           info      = ''
###[ 802.11 Rates ]### 
           ID        = 1
           len       = 4
           rates     = [0x2, 0x4, 0xb, 0x16]
###[ 802.11 Information Element ]### 
           ID        = ESRates
           len       = 8
           info      = '\x0c\x12\x18$0H`l'
###[ 802.11 Information Element ]### 
           ID    